In [3]:
import subprocess
import pandas as pd

In [41]:
model_name = 'models/26jul_E'
model_data = 'models-data/data_26jul_1.dat'

In [42]:
p = subprocess.Popen(['C:/Program Files (x86)/biogeme-2.4/biogeme-2.4/biogeme.exe', model_name,model_data], shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
for line in p.stdout.readlines():
    print (line),
retval = p.wait()

b'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\r\n'
b'biogeme 2.4 [Sun Oct 25 23:01:47 WEST 2015]\r\n'
b'Michel Bierlaire, EPFL\r\n'
b'-- Compiled by biogeme@hotmail.com on MINGW32_NT-6.1\r\n'
b'See http://biogeme.epfl.ch\r\n'
b'                    !! CFSQP is available !!\r\n'
b'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\r\n'
b'\t"In every non-trivial program there is at least one bug."\r\n'
b'\r\n'
b'\r\n'
b'[10:48:50]patFileNames.cc:52  models/26jul_E.par does not exist\r\n'
b'[10:48:50]patFileNames.cc:56  Trying default.par instead\r\n'
b'[10:48:50]patBiogeme.cc:134  Read default.par\r\n'
b' Opening file models-data/data_26jul_1.dat\r\n'
b' Data  file... line 500\tMemory: 67 Kb\r\n'
b' Total obs.:   837\r\n'
b' Total memory: 112.934 Kb\r\n'
b' Run time for data processing: 00:00\r\n'
b' Init loglike=-985.941\r\n'
b'     gmax Iter   radius        f(x)     Status       rhok nFree\r\n'
b' +2.39e-001    1 1.00e+000 +9.8594093e+002 ****OutTrReg +9.77e-001 7  ++ P\r

In [43]:
with open(model_name+'_res.mod','wb') as simmod:
    with open(model_name+'.res','rb') as res:
        searchlines = res.readlines()
        foundit = False
        for i, line in enumerate(searchlines):
            if foundit==False:
                if b'[SampleEnum]' in line:
                    print('Found it.')
                    foundit = True
                simmod.write(line)
            else:
                if line.decode("utf-8").find('0\r\n')==0:
                    print('Changed to 100.')
                    simmod.write(b'100\r\n')
                    foundit==False
                else:
                    simmod.write(line)

Found it.
Changed to 100.


In [44]:
p = subprocess.Popen(['C:/Program Files (x86)/biogeme-2.4/biogeme-2.4/biosim.exe', model_name+'_res',model_data], shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
for line in p.stdout.readlines():
    print (line),
retval = p.wait()

b'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\r\n'
b'biogeme 2.4 [Sun Oct 25 23:01:47 WEST 2015]\r\n'
b'Michel Bierlaire, EPFL\r\n'
b'-- Compiled by biogeme@hotmail.com on MINGW32_NT-6.1\r\n'
b'See http://biogeme.epfl.ch\r\n'
b'                    !! CFSQP is available !!\r\n'
b'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\r\n'
b'\t"In every non-trivial program there is at least one bug."\r\n'
b'\r\n'
b'\r\n'
b'[10:48:54]patFileNames.cc:52  models/26jul_E_res.par does not exist\r\n'
b'[10:48:54]patFileNames.cc:56  Trying default.par instead\r\n'
b'[10:48:54]patBiogeme.cc:134  Read default.par\r\n'
b'Warning:  Lower bound on mu set to 1e-005\r\n'
b'Warning:   Value defined by gevMinimumMu in default.par\r\n'
b' Opening file models-data/data_26jul_1.dat\r\n'
b' Data  file... line 500\tMemory: 67 Kb\r\n'
b' Total obs.:   837\r\n'
b' Total memory: 112.934 Kb\r\n'
b' 59%\t[500/838] Est. term. time: 07/26/16 10:48:54 (in 00:00)\r\n'
b' BIOSIM Input file\r\n'
b' =========

In [45]:
simres = pd.read_csv(model_name+'_res.enu',sep='\t')
simres.head()

,Choice_Id,P_choice,V_A1_TAXI,V_A2_TRANSIT,V_A3_OTHER,V_A4_FREE,P_A1_TAXI,Residual_A1_TAXI,P_A2_TRANSIT,Residual_A2_TRANSIT,...,Simul91,Simul92,Simul93,Simul94,Simul95,Simul96,Simul97,Simul98,Simul99,Simul100
0,1,0.529882,-0.442943,-1.314230,-1.20052,-9999.99,0.529882,0.470118,0.221709,-0.221709,...,2,2,1,3,1,1,2,1,3,1
1,1,0.529882,-0.442943,-1.314230,-1.20052,-9999.99,0.529882,0.470118,0.221709,-0.221709,...,1,1,1,1,1,1,1,1,3,3
2,1,0.529882,-0.442943,-1.314230,-1.20052,-9999.99,0.529882,0.470118,0.221709,-0.221709,...,3,3,1,2,1,2,3,1,3,1
3,1,0.529882,-0.442943,-1.314230,-1.20052,-9999.99,0.529882,0.470118,0.221709,-0.221709,...,1,1,2,1,2,2,3,2,2,3
4,1,0.466793,-0.877370,-1.748657,-1.20052,-9999.99,0.466793,0.533207,0.195312,-0.195312,...,2,3,1,2,1,1,2,1,3,1


In [46]:
simul_start = list(simres.columns).index('Simul1')
from collections import Counter
def count_correct(row):
    return list(row[simul_start:]).count(row.Choice_Id)
def simul_mode(row):
    return int(Counter(list(row[simul_start:])).most_common(1)[0][0])

In [47]:
simres['Accuracy'] = simres.apply(count_correct,axis=1)
simres['Mode_Prediction'] = simres.apply(simul_mode,axis=1)

In [48]:
simres['ModeAccurate'] = simres['Choice_Id']==simres['Mode_Prediction']

In [49]:
simres.groupby('Choice_Id')['Accuracy'].mean()

Choice_Id
1    58.708571
2    21.050420
3    24.492958
4    99.991189
Name: Accuracy, dtype: float64

In [50]:
simres.groupby('Choice_Id')['ModeAccurate'].mean()

Choice_Id
1    0.982857
2    0.067227
3    0.014085
4    1.000000
Name: ModeAccurate, dtype: float64

In [51]:
simres.groupby(['Choice_Id','Mode_Prediction']).size()

Choice_Id  Mode_Prediction
1          1                  344
           2                    6
2          1                  109
           2                    8
           3                    2
3          1                  137
           2                    3
           3                    2
4          4                  227
dtype: int64